# Introduction: How many years of our life are worth living?
---

**Health-adjusted life expectancy (HALE)** is the average number of years that a person can expect to live in full health—that is, not hampered by disabling illnesses or injuries. HALE estimates are used to predict future health service needs, evaluate existing health programs, and determine standards of living. 

While governments devote substantial resources into increasing the metric of life expectancy, higher life expectancy does not necessarily lead to higher wellbeing overall without a similar increase in HALE. For instance, while projected life expectancy seems high in developed countries, the actual number of years lived to the fullest can be substantially lower when taking into account that large percentage of elderlies are ill or bedridden.

Our group intends to compare how life expectancy and HALE are related, as well as factors that affects HALE that may inform future policy decisions.

### Essential Libraries

We begin by importing the essential Python Libraries.

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
from matplotlib.pyplot import figure

sb.set() # set the default Seaborn style for graphics

#Libraires to read multiple csv files
import os
import glob

import pickle # to save dataframe

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Data Extraction
---
### Datasets used

Global Health Data Exchange (GHDx) datasets (https://ghdx.healthdata.org/):
- **hale.csv** : Health-Adjusted Lide Expectancy (HALE) of all countries and territories from years 2000-2019. 

World Bank datasets (https://data.worldbank.org/):
- **% basic sanitation services.csv** : Percentage of population with access to basic sanitation services of all countries from years 2000-2019.
- **age dependency ratio.csv** : Age-dependency ratio (ratio of number of dependents e.g. children and elderly to number of workers) as percentage of working-age population of all countries from years 2000-2019.
- **c02 emissions mtpc.csv** : CO2 emissions (metric tons per capita) of all countries from years 2000-2019.
- **domestic private health expenditure.csv** : Domestic private health expenditure as a percentage of current health expenditure of all countries from years 2000-2019.
- **gdppc.csv** : Gross Domestic Product (GDP) per capita of all countries from years 2000-2019.
- **health expenditure %gdp.csv** : Health expenditure as a percentage of GDP of all countries from years 2000-2019.
- **immunisation measles.csv** : Percentage of children aged 12-23 months who received measles vaccination for all countries from years 2000-2019.
- **life expectancy.csv** : Average life expectancy of all countries from years 2000-2019.
- **population density.csv** : Population desity of all countries from years 2000-2019.
- **population.csv** : Population of all countries from years 2000-2019.
---

### Reading the .csv files

Because we took the HALE dataset from a different source compared to the other datasets, the formatting of the data in the csv files are different. Hence we implement code to read both differently.

In [2]:
df = pd.read_csv('hale.csv') # dataset from GHDx
df.head()

,measure,location,sex,age,metric,year,val,upper,lower
0,HALE (Healthy life expectancy),Afghanistan,Both,All Ages,Years,2000,46.622245,49.847779,42.686020
1,HALE (Healthy life expectancy),Afghanistan,Both,All Ages,Years,2001,46.745406,50.067600,42.797207
2,HALE (Healthy life expectancy),Afghanistan,Both,All Ages,Years,2002,47.385002,50.781360,43.562080
3,HALE (Healthy life expectancy),Afghanistan,Both,All Ages,Years,2003,48.227751,51.634578,44.516347
4,HALE (Healthy life expectancy),Afghanistan,Both,All Ages,Years,2004,48.798730,52.120954,45.169141


In [3]:
df = pd.read_csv('% basic sanitation services.csv', skiprows=4) # dataset from World Bank
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,People using at least basic sanitation service...,SH.STA.BASS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,97.543481,97.543481,97.543481,97.543481,97.543481,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,People using at least basic sanitation service...,SH.STA.BASS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,27.684665,28.166121,28.648935,29.134289,29.618405,30.114592,30.384759,30.742610,30.931064,NaN
2,Afghanistan,AFG,People using at least basic sanitation service...,SH.STA.BASS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,37.960809,39.506018,41.060966,42.626023,44.201217,45.789127,47.390662,49.006170,50.499647,NaN
3,Africa Western and Central,AFW,People using at least basic sanitation service...,SH.STA.BASS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,29.556282,30.278810,31.012072,31.753439,32.506339,33.270972,33.921662,34.684126,35.437188,NaN
4,Angola,AGO,People using at least basic sanitation service...,SH.STA.BASS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,43.591472,44.854793,46.117029,47.374481,48.627945,49.876980,51.120744,51.393624,51.660329,NaN


In [4]:
# read all data
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))
csv_files = sorted(csv_files)
dfs = []
# loop over the list of csv files
for f in csv_files:
    # read the csv file
    try:
        # read HALE data
        df = pd.read_csv(f)
        df.rename(columns={'val':'HALE', 'location':'Country Name'}, inplace=True)
        df=df[['Country Name','year', 'HALE']]
        df = pd.pivot_table(df, index=df['Country Name'],columns=df['year'], values='HALE').reset_index().rename_axis(None, axis=1) # pivot year rows to become columns
        df.insert(1, 'Indicator Name', 'HALE')
        df.columns = df.columns.astype(str)
    except:
        # read worldbank data
        df = pd.read_csv(f, skiprows=4) # world bank data has to be read differently, first 4 rows are skipped
        temp1 = df[df.columns[df.columns.isin(['Country Name', 'Indicator Name'])]]
        temp2 = df[df.columns[df.columns.isin(str(x) for x in range(2000,2020))]] # include only years 2000-2019
        df = pd.concat([temp1, temp2], axis=1)
    dfs.append(df)

# print out variable names of the dataframes saved in dfs
for df in dfs:
    print(df['Indicator Name'][0])

People using at least basic sanitation services (% of population)
CO2 emissions (metric tons per capita)
Age dependency ratio (% of working-age population)
Domestic private health expenditure (% of current health expenditure)
GDP per capita (current US$)
HALE
Current health expenditure (% of GDP)
Immunization, measles (% of children ages 12-23 months)
Immunization, DPT (% of children ages 12-23 months)
Life expectancy at birth, total (years)
Population density (people per sq. km of land area)
Population, total


In [5]:
dfs[0].head()

,Country Name,Indicator Name,2000,2001,2002,2003,2004,2005,2006,2007,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,People using at least basic sanitation service...,98.320210,98.255483,98.190755,98.126028,98.061300,97.996573,97.931846,97.867118,...,97.672936,97.608209,97.543481,97.543481,97.543481,97.543481,97.543481,NaN,NaN,NaN
1,Africa Eastern and Southern,People using at least basic sanitation service...,22.766523,23.180937,23.579701,23.968289,24.350744,24.808894,25.252879,25.677641,...,27.109043,27.205747,27.684665,28.166121,28.648935,29.134289,29.618405,30.114592,30.384759,30.742610
2,Afghanistan,People using at least basic sanitation service...,21.870802,21.881304,23.274193,24.667435,26.067695,27.475416,28.884040,30.374641,...,34.898847,36.425149,37.960809,39.506018,41.060966,42.626023,44.201217,45.789127,47.390662,49.006170
3,Africa Western and Central,People using at least basic sanitation service...,22.335506,22.569922,23.106871,23.675746,24.262708,24.877718,25.508586,26.149466,...,28.150506,28.846399,29.556282,30.278810,31.012072,31.753439,32.506339,33.270972,33.921662,34.684126
4,Angola,People using at least basic sanitation service...,27.557523,28.989223,30.422467,31.856061,33.290391,34.722639,35.991281,37.259306,...,41.060793,42.326664,43.591472,44.854793,46.117029,47.374481,48.627945,49.876980,51.120744,51.393624


# Data Cleaning
---
### Removing Rows with Missing Values

We remove the rows in the dataframes that have more than half of their data as NaN values. 

In [6]:
def find(s, el):
    for i in s.index:
        if s[i] == el: 
            return i
    return None

In [7]:
#drop rows with more than half of the columns having NaN values for all the datasets
i = 0 
for i in range(len(dfs)):
  dfs[i] = dfs[i].dropna(thresh = 9)
  dfs[i].reset_index()

### Standardising Countries in Dataframes

In order to keep the countries used uniform across all dataframes, if a country would be removed from a dataframe, it would also be removed across all dataframes.

In [8]:
# choose dataframe with the largest number of countries and compare it to others to find out which countries have all valid values
df_sizes = [len(df.index) for df in dfs]
df_largestsize = df_sizes.index(max(df_sizes))
country_name_df = pd.DataFrame(dfs[df_largestsize]["Country Name"])

countlist = [0 for x in range(len(country_name_df))]
country_name_df["Count"] = countlist
country_name_df

,Country Name,Count
0,Aruba,0
1,Africa Eastern and Southern,0
2,Afghanistan,0
3,Africa Western and Central,0
4,Angola,0
...,...,...
261,Kosovo,0
262,"Yemen, Rep.",0
263,South Africa,0
264,Zambia,0


In [9]:
# the 110th country in the dataframe is labelled as "Not Classified" so we skip it
for df in dfs:
  i=0
  for country in country_name_df["Country Name"]:
    if country in df["Country Name"].tolist(): 
        country_name_df["Count"][i]+=1
        
    if(i!=109):
      i+=1
    else:
      i+=2
country_name_df["Count"]

0       6
1      11
2      12
3      11
4      12
       ..
261     4
262    11
263    12
264    12
265    12
Name: Count, Length: 265, dtype: int64

In [10]:
# To do data cleaning, we select only the countries that appear in all datasets. We delete the rest of the data. Finally, have 11 datasets and each contains 158 countries.
for df in dfs:
  i=0
  truelist = ["False" for x in range(len(df))]
  df["True List"] = truelist
  for country in df["Country Name"]:
    if country in country_name_df[country_name_df["Count"]==len(dfs)]["Country Name"].tolist():
      df["True List"][find(df["Country Name"],country)]=True
    i+=1
  
i=0
for i in range(len(dfs)):
  dfs[i]=dfs[i].loc[dfs[i]['True List']==True]

for i in range(len(dfs)):
  dfs[i].drop("True List", axis=1, inplace=True)
  dfs[i].reset_index(inplace=True, drop=True)

len(dfs[0])

158

### Error Correction

We fill in the remaining missing values in each row by first back filling empty cells with the values that come after then, then forward filling remaining empty cells with values that come before them (this would apply to the last cells in the row). The reason for using back filling and forward filling is because the missing data would probably follow the trends of the data over the years


We decided not to remove any outliers in the datasets. This is because there are a lot of variation in countries, so large variations in the data would be expected. It would also mean removing a lot of valuable data across all the dataframes.

In [11]:
#For those missing vales, we do error correction: To backfill the missing values, then forward fill any remaining missing values at the front
for i in range(len(dfs)):
    dfs[i] = dfs[i].bfill(axis ='columns')
    dfs[i] = dfs[i].ffill(axis ='columns')

### Standardising the Dataframes

We now convert the dataframes into standardised formats with rows as the years and columns as the countries. We also remove redundant columns or columns with extraneous information such as Country Code, Indicator Name, Indicator Code etc.

In [12]:
# put all dfs in a df dictionary
dfs_dict = {}
for df in dfs:
    dfs_dict[df['Indicator Name'][0]] = df;
    df.drop('Indicator Name', axis=1, inplace=True)
print(dfs_dict.keys())

dict_keys(['People using at least basic sanitation services (% of population)', 'CO2 emissions (metric tons per capita)', 'Age dependency ratio (% of working-age population)', 'Domestic private health expenditure (% of current health expenditure)', 'GDP per capita (current US$)', 'HALE', 'Current health expenditure (% of GDP)', 'Immunization, measles (% of children ages 12-23 months)', 'Immunization, DPT (% of children ages 12-23 months)', 'Life expectancy at birth, total (years)', 'Population density (people per sq. km of land area)', 'Population, total'])


In [13]:
def transform_df(dfs_dict, name):
    '''transform a dataframe with a given name to the format convenient for machine learning
    :param dfs_dict: a dictionary containing dataframes
    :param name: name of the dataframe to be extracted and transformed
    :returns: transformed dataframe 
    '''
    
    df = dfs_dict[name] #extract df to be transformed
    result_df = pd.DataFrame(columns = ["Country Name", name]) #initialize the result df
    result_df.set_index("Country Name", inplace = True)

    for i in range(df.shape[0]): #do for each country in the data frame 
        series = df.iloc[i] #extract a time-series for a country

        #set up the temp df
        country_name = series.values[0] 
        temp_df = pd.DataFrame({'Country Name' : series.index, name : series.values})
        temp_df.drop(axis = 0, index = 0, inplace = True) #drop the row with irrelevant data
        temp_df.set_index("Country Name", inplace = True) #set "Country Name" as new index

        #rename the year column to Country - Year format
        for j in temp_df.index:
            new_name = country_name + ' - ' + j
            temp_df.rename(index = {j : new_name}, inplace = True)
        result_df = result_df.append(temp_df) #add to result array
    return result_df.sort_index()

In [14]:
#now we transform each of our datasets to revelant format 
hale_df = transform_df(dfs_dict, "HALE")
sanitation_df = transform_df(dfs_dict, "People using at least basic sanitation services (% of population)")
co2_df = transform_df(dfs_dict, 'CO2 emissions (metric tons per capita)')
age_dependency_df = transform_df(dfs_dict,'Age dependency ratio (% of working-age population)')
le_df = transform_df(dfs_dict, 'Life expectancy at birth, total (years)')
current_health_df = transform_df(dfs_dict, 'Current health expenditure (% of GDP)')
domestic_private_health_df = transform_df(dfs_dict, 'Domestic private health expenditure (% of current health expenditure)')
GDP_per_capital_df = transform_df(dfs_dict, 'GDP per capita (current US$)')
measles_df = transform_df(dfs_dict, 'Immunization, measles (% of children ages 12-23 months)')
pop_total_df = transform_df(dfs_dict, 'Population, total')
pop_density_df = transform_df(dfs_dict, 'Population density (people per sq. km of land area)')

### Combining all Dataframes

We now combine all dataframes into a single dataframe, finalised_df, with the rows as countries and the columns as metrics (including year).

In [15]:
to_concat = [hale_df,le_df,sanitation_df,co2_df,age_dependency_df,current_health_df,domestic_private_health_df,GDP_per_capital_df,measles_df,pop_density_df,pop_total_df]
finalized_df = pd.concat(to_concat, axis = 1)
finalized_df

,HALE,"Life expectancy at birth, total (years)",People using at least basic sanitation services (% of population),CO2 emissions (metric tons per capita),Age dependency ratio (% of working-age population),Current health expenditure (% of GDP),Domestic private health expenditure (% of current health expenditure),GDP per capita (current US$),"Immunization, measles (% of children ages 12-23 months)",Population density (people per sq. km of land area),"Population, total"
Country Name,,,,,,,,,,,
Afghanistan - 2000,46.622245,55.841,21.870802,0.037055,104.856398,9.44339,85.37561,179.426579,27.0,31.829117,20779957.0
Afghanistan - 2001,46.745406,56.308,21.881304,0.037488,104.476104,9.44339,85.37561,179.426579,37.0,33.095904,21606992.0
Afghanistan - 2002,47.385002,56.784,23.274193,0.048671,103.671183,9.44339,85.37561,179.426579,35.0,34.618102,22600774.0
Afghanistan - 2003,48.227751,57.271,24.667435,0.057008,102.663238,8.941258,86.069191,190.683814,39.0,36.27251,23680871.0
Afghanistan - 2004,48.79873,57.772,26.067695,0.0457,101.562371,9.808474,84.527596,211.382074,48.0,37.874413,24726689.0
...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe - 2015,51.412576,59.534,38.120646,0.897598,83.13859,7.452066,54.925795,1445.069702,86.0,35.710591,13814642.0
Zimbabwe - 2016,51.990322,60.294,37.529193,0.783303,83.576729,7.675163,48.78302,1464.588957,95.0,36.268161,14030338.0
Zimbabwe - 2017,52.545069,60.812,36.941673,0.71857,83.466245,7.469752,45.278132,1235.189032,90.0,36.801342,14236599.0


In [16]:
# save finalized_df and file so they can be used in other notebooks
finalized_df = finalized_df.astype(float, errors = 'raise')
finalized_df.to_pickle("finalized_df.pkl")

# save dictionary of metrics dataframes
np.save('dfs_dict.npy', dfs_dict)